In [1]:
import pandas as pd
import numpy as np
import nbprogress
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from datetime import datetime

In [2]:
# Import our file
df = pd.read_csv('data/df_webscraping_2020-04-13-111815.csv', header = 0, encoding = 'utf-8', sep = ',')
df.head(5)

,debunk_id,hoax_link,true_false
0,1,http://www.paulomouvementcitoyen.com/2017/02/u...,FAUX
1,1,https://www.blog.sami-aldeeb.com/2017/02/06/pr...,FAUX
2,1,https://francaisdefrance.wordpress.com/2016/12...,FAUX
3,1,https://www.lemonde.fr/les-decodeurs/article/2...,VRAI
4,2,https://www.buzzbeed.com/macron-veut-faire-pay...,FAUX


In [3]:
# Here, we add the columns which we'll populate with the web scraper
df['title'] = np.nan
df['body'] = np.nan
df['sources'] = np.nan

In [4]:
def webscrape_decodex(df) :
    
    driver_path = 'app/chromedriver.exe'
    brave_path = 'C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe'

    chrome_options = Options()
    chrome_options.binary_location = brave_path
    
    row_number = np.where(pd.isnull(df['body']))[0][0]
    erreurs = 0
    
    for idx, row in nbprogress.log(df.iterrows(), size = len(df)) :
        if idx < row_number :
            pass
        else :
            try :
                browser = webdriver.Chrome(executable_path = driver_path, options = chrome_options)
                url = row['hoax_link']
                browser.get(url)
                html_source = browser.page_source
                browser.quit()
                soup = BeautifulSoup(html_source, 'html.parser')
                
                #####################
                ### Title Section ###
                #####################
                
                try :
                    title = soup.find('h1').text.strip()
                except :
                    title = ''
                try :
                    subtitle = soup.find('h2').text.strip()
                except :
                    subtitle = ''
        
                complete_title = title + subtitle
                
                ############################
                ### Text Content Section ###
                ############################
                
                text = ''
                for p_tag in soup.find_all('p') :
                    text = text + ' ' + p_tag.text.strip()
                text = text.strip()
                    
                ########################
                ### URL List Section ###
                ########################
                
                url_list = ''
                for link in soup.body.find_all('a', href=True):
                    if re.search(r'http', link['href']) :
                        url_list = link['href'] + ', '
                url_list = url_list.strip()
                url_list = url_list[:-1]
                        
                # Finally, we append those values in our dataframe
                
                df.loc[row_number, 'title'] = complete_title
                df.loc[row_number, 'body'] = text
                df.loc[row_number, 'sources'] = url_list
                
                row_number += 1
                
            except :
                df.loc[row_number, 'title'] = 'Connection refused'
                df.loc[row_number, 'body'] = 'Connection refused'
                df.loc[row_number, 'sources'] = 'Connection refused'
                
                erreurs += 1
                
    print('Nombre d\'erreurs rencontrées pendant le webscraping : ', erreurs)
    
    filename = './data/webscraped_data_' + datetime.now().strftime("%Y-%m-%d-%H%M%S.json")
    df.to_json(filename)
    # Then open the file with pd.read_json (r'Path where you saved the JSON file\File Name.json')

In [5]:
webscrape_decodex(df)

Nombre d'erreurs rencontrées pendant le webscraping :  12


NameError: name 'datetime' is not defined

In [6]:
df.to_json('./data/webscraped_data.json')